In [15]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
# Create a base class for declarating class definitions to produce Table objects
Base = declarative_base()

/var/folders/d2/rs9l0yts25jd2cswktf_sr9c0000gn/T/ipykernel_64492/3836882636.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [16]:
class FantasyFootball(Base):
    __tablename__ = 'fantasydata'
    
    id = Column(Integer, primary_key=True)
    Week = Column(Float)
    Name = Column(String)
    Position = Column(String)
    Year = Column(Integer)
    Game = Column(String)
    FantasyPoints = Column(Integer)
    PassingYards = Column(Float)
    PassingTDs = Column(Float)
    Ints = Column(Integer)
    RushAttempts = Column(Integer)
    RushYards = Column(Integer)
    Fumbles = Column(Integer)
    RushTDs = Column(Float)
    Receptions = Column(Float)
    ReceivingYards = Column(Float)
    ReceivingTDs = Column(Float)

In [17]:
engine = create_engine('sqlite:///football.db')
Base.metadata.create_all(engine)

In [18]:
from sqlalchemy.orm import Session
from sqlalchemy import func
session = Session(bind=engine)

In [19]:
results = session.query(FantasyFootball.Name, FantasyFootball.Year, func.sum(FantasyFootball.FantasyPoints).label('Fantasy_Points'),func.sum(FantasyFootball.RushTDs).label('Rushing_TDs'),func.sum(FantasyFootball.ReceivingTDs).label('Receiving_TDs'),func.sum(FantasyFootball.Receptions).label('Receptions')) \
                 .filter(FantasyFootball.Position == 'RB') \
                 .group_by(FantasyFootball.Name, FantasyFootball.Year).order_by(FantasyFootball.Name,FantasyFootball.Year) \
                 .all()

In [20]:
results

[('AJ Dillon', '2020', 36.0, 2, 0.0, 2.0),
 ('AJ Dillon', '2021', 172.0, 5, 2.0, 34.0),
 ('AJ Dillon', '2022', 157.0, 7, 0.0, 28.0),
 ('AJ Dillon', '2023', 108.0, 2, 0.0, 22.0),
 ('Aaron Jones', '2019', 304.0, 16, 3.0, 49.0),
 ('Aaron Jones', '2020', 247.0, 9, 2.0, 47.0),
 ('Aaron Jones', '2021', 218.0, 4, 6.0, 52.0),
 ('Aaron Jones', '2022', 237.0, 2, 5.0, 59.0),
 ('Aaron Jones', '2023', 127.0, 2, 1.0, 30.0),
 ('Adam Prentice', '2021', 4.0, 0, 0.0, 3.0),
 ('Adam Prentice', '2022', 3.0, 0, 0.0, 3.0),
 ('Adam Prentice', '2023', 1.0, 0, 0.0, 2.0),
 ('Adrian Killins', '2020', 1.0, 0, 0.0, 1.0),
 ('Adrian Peterson', '2019', 135.0, 5, 0.0, 17.0),
 ('Adrian Peterson', '2020', 114.0, 7, 0.0, 12.0),
 ('Adrian Peterson', '2021', 25.0, 2, 0.0, 4.0),
 ('Alec Ingold', '2019', 15.0, 0, 1.0, 6.0),
 ('Alec Ingold', '2020', 26.0, 0, 1.0, 12.0),
 ('Alec Ingold', '2021', 22.0, 0, 1.0, 10.0),
 ('Alec Ingold', '2022', 34.0, 1, 1.0, 15.0),
 ('Alec Ingold', '2023', 22.0, 0, 0.0, 13.0),
 ('Alex Armah', '2019

In [21]:
data_dict = {}

# Iterate over the list of tuples
for name, year, fantasy_points, RushTDs, ReceivingTDs, Receptions in results:
    # If the name is not already a key in the dictionary, add it with an empty dictionary as its value
    if name not in data_dict:
        data_dict[name] = {}
    
    # Add or update the number as key and its corresponding value
    data_dict[name][year] = {
        'Fantasy_Points': fantasy_points,
        'Rush_TD_Points': RushTDs * 6,
        'Receiving_TD_Points': ReceivingTDs * 6,
        'Reception_Points': Receptions * 1
    }# Replace `value` with the actual value you want to associate with the number

# Print the resulting dictionary
print(data_dict)

{'AJ Dillon': {'2020': {'Fantasy_Points': 36.0, 'Rush_TD_Points': 12, 'Receiving_TD_Points': 0.0, 'Reception_Points': 2.0}, '2021': {'Fantasy_Points': 172.0, 'Rush_TD_Points': 30, 'Receiving_TD_Points': 12.0, 'Reception_Points': 34.0}, '2022': {'Fantasy_Points': 157.0, 'Rush_TD_Points': 42, 'Receiving_TD_Points': 0.0, 'Reception_Points': 28.0}, '2023': {'Fantasy_Points': 108.0, 'Rush_TD_Points': 12, 'Receiving_TD_Points': 0.0, 'Reception_Points': 22.0}}, 'Aaron Jones': {'2019': {'Fantasy_Points': 304.0, 'Rush_TD_Points': 96, 'Receiving_TD_Points': 18.0, 'Reception_Points': 49.0}, '2020': {'Fantasy_Points': 247.0, 'Rush_TD_Points': 54, 'Receiving_TD_Points': 12.0, 'Reception_Points': 47.0}, '2021': {'Fantasy_Points': 218.0, 'Rush_TD_Points': 24, 'Receiving_TD_Points': 36.0, 'Reception_Points': 52.0}, '2022': {'Fantasy_Points': 237.0, 'Rush_TD_Points': 12, 'Receiving_TD_Points': 30.0, 'Reception_Points': 59.0}, '2023': {'Fantasy_Points': 127.0, 'Rush_TD_Points': 12, 'Receiving_TD_Points'

In [22]:
name_values_dict = {}
years = ['2019','2020','2021','2022','2023']

# Iterate over the data dictionaries
for name, nested_dict in data_dict.items():
    # Extract values and add them to the list
    values_list = [nested_dict.get(year, {'Fantasy_Points': 0, 'Rush_TD_Points' : 0, 'Receiving_TD_Points': 0, 'Reception_Points': 0}) for year in years]
    # Add the list of values to the dictionary
    name_values_dict[name] = values_list

# Print the resulting dictionary
print(name_values_dict)

{'AJ Dillon': [{'Fantasy_Points': 0, 'Rush_TD_Points': 0, 'Receiving_TD_Points': 0, 'Reception_Points': 0}, {'Fantasy_Points': 36.0, 'Rush_TD_Points': 12, 'Receiving_TD_Points': 0.0, 'Reception_Points': 2.0}, {'Fantasy_Points': 172.0, 'Rush_TD_Points': 30, 'Receiving_TD_Points': 12.0, 'Reception_Points': 34.0}, {'Fantasy_Points': 157.0, 'Rush_TD_Points': 42, 'Receiving_TD_Points': 0.0, 'Reception_Points': 28.0}, {'Fantasy_Points': 108.0, 'Rush_TD_Points': 12, 'Receiving_TD_Points': 0.0, 'Reception_Points': 22.0}], 'Aaron Jones': [{'Fantasy_Points': 304.0, 'Rush_TD_Points': 96, 'Receiving_TD_Points': 18.0, 'Reception_Points': 49.0}, {'Fantasy_Points': 247.0, 'Rush_TD_Points': 54, 'Receiving_TD_Points': 12.0, 'Reception_Points': 47.0}, {'Fantasy_Points': 218.0, 'Rush_TD_Points': 24, 'Receiving_TD_Points': 36.0, 'Reception_Points': 52.0}, {'Fantasy_Points': 237.0, 'Rush_TD_Points': 12, 'Receiving_TD_Points': 30.0, 'Reception_Points': 59.0}, {'Fantasy_Points': 127.0, 'Rush_TD_Points': 12, 

In [23]:
name_values_dict

{'AJ Dillon': [{'Fantasy_Points': 0,
   'Rush_TD_Points': 0,
   'Receiving_TD_Points': 0,
   'Reception_Points': 0},
  {'Fantasy_Points': 36.0,
   'Rush_TD_Points': 12,
   'Receiving_TD_Points': 0.0,
   'Reception_Points': 2.0},
  {'Fantasy_Points': 172.0,
   'Rush_TD_Points': 30,
   'Receiving_TD_Points': 12.0,
   'Reception_Points': 34.0},
  {'Fantasy_Points': 157.0,
   'Rush_TD_Points': 42,
   'Receiving_TD_Points': 0.0,
   'Reception_Points': 28.0},
  {'Fantasy_Points': 108.0,
   'Rush_TD_Points': 12,
   'Receiving_TD_Points': 0.0,
   'Reception_Points': 22.0}],
 'Aaron Jones': [{'Fantasy_Points': 304.0,
   'Rush_TD_Points': 96,
   'Receiving_TD_Points': 18.0,
   'Reception_Points': 49.0},
  {'Fantasy_Points': 247.0,
   'Rush_TD_Points': 54,
   'Receiving_TD_Points': 12.0,
   'Reception_Points': 47.0},
  {'Fantasy_Points': 218.0,
   'Rush_TD_Points': 24,
   'Receiving_TD_Points': 36.0,
   'Reception_Points': 52.0},
  {'Fantasy_Points': 237.0,
   'Rush_TD_Points': 12,
   'Receiving_

In [24]:
import json

# Convert the dictionary to JSON format
json_data = json.dumps(name_values_dict, indent=4)

# Write the JSON data to a file
with open('../../Flask/Static/RBYear.json','w') as json_file:
    json_file.write(json_data)